# <a id='toc1_'></a>[Loan Default Prediction (Preprocessing)](#toc0_)

**Table of contents**<a id='toc0_'></a>    
- [Loan Default Prediction (Preprocessing)](#toc1_)    
  - [Libraries](#toc1_1_)    
  - [Read](#toc1_2_)    
  - [Preprocessing](#toc1_3_)    
    - [Split dataset](#toc1_3_1_)    
    - [Feature selection](#toc1_3_2_)    
      - [Unwanted features](#toc1_3_2_1_)    
      - [Null rates](#toc1_3_2_2_)    
      - [Duplicates features](#toc1_3_2_3_)    
      - [Constant and Quasi-constant features](#toc1_3_2_4_)    
      - [Correlated features](#toc1_3_2_5_)    
    - [Transform](#toc1_3_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Libraries](#toc0_)

In [167]:
# Basic python
import re
import sys

# Data manipulation
import polars as pl
import pandas as pd
import datetime as dt

# Preprocessing
from feature_engine.selection import DropFeatures, DropConstantFeatures, DropDuplicateFeatures, DropCorrelatedFeatures

# appending a path
sys.path.append('../src/')

# Own modules
import helpers as hp

## <a id='toc1_2_'></a>[Read](#toc0_)

In [168]:
# Define file path
path = "../data/raw/"
raw_file = "definitely_not_from_kaggle_loan_default_dataset.csv"

In [169]:
# Read data
df = pl.read_csv(path + raw_file)

In [170]:
df.head(3)

uuid,credit_application_date,default,person_bith_year_month_1_no_aggregation,activity_date_36_month_1_no_aggregation,location_cluster_03_month_1_most_frequent,contract_type_1_month_1_no_aggregation,activity_pattern_35_month_1_no_aggregation,activity_pattern_34_month_12_no_aggregation,activity_pattern_17_month_12_no_aggregation,activity_pattern_02_month_12_no_aggregation,device_age_2_month_1_social_mean,activity_pattern_09_last_12_months_max,client_type_1_month_9_no_aggregation,activity_pattern_07_last_9_months_max,activity_pattern_07_last_12_months_max,line_balance_type_11_last_12_months_max,line_balance_type_09_last_9_months_mean,activity_pattern_21_weekend_month_12_no_aggregation,activity_pattern_09_last_6_months_max,activity_pattern_09_last_9_months_max,line_balance_type_09_last_12_months_mean,activity_pattern_17_month_9_no_aggregation,line_balance_type_01_last_3_months_min,plan_data_month_12_no_aggregation,activity_pattern_25_last_12_months_min,data_usage_pattern_fortnight_1_last_12_months_max,location_work_null_month_9_no_aggregation,line_balance_type_01_last_6_months_min,activity_pattern_21_weekdaynight_month_12_no_aggregation,activity_pattern_03_weekend_month_6_no_aggregation,activity_pattern_25_last_9_months_min,mobility_pattern_03_last_3_months_mean,data_usage_pattern_days_last_6_months_min,activity_pattern_02_month_1_no_aggregation,activity_pattern_02_last_12_months_mean,billing_pattern_03_last_12_months_mean,…,activity_pattern_14_last_12_months_mean,activity_pattern_12_last_12_months_mean,activity_pattern_05_weekdaylight_last_3_months_mean,activity_pattern_14_last_9_months_mean,device_age_3_month_12_no_aggregation,credit_default_home_last_12_months_min,activity_pattern_14_last_6_months_max,device_technology_2_month_1_no_aggregation,activity_pattern_11_month_12_no_aggregation,device_count_1_month_1_no_aggregation,activity_pattern_27_last_9_months_mean,activity_pattern_21_weekdaynight_last_12_months_max,data_usage_pattern_time_8_month_1_social_min,line_balance_type_04_last_6_months_max,mobility_pattern_05_month_6_no_aggregation,data_usage_pattern_time_5_month_1_social_mean,data_usage_pattern_week_1_last_9_months_mean,location_cluster_02_month_9_no_aggregation,credit_default_work_last_6_months_mean,line_balance_type_07_last_9_months_min,data_usage_pattern_fortnight_1_last_9_months_mean,activity_pattern_11_last_9_months_max,activity_pattern_10_month_6_no_aggregation,activity_pattern_10_month_9_no_aggregation,line_months_active_2_last_12_months_mean,data_usage_pattern_days_last_3_months_mean,activity_pattern_10_weekend_last_3_months_mean,data_usage_pattern_fortnight_4_last_6_months_max,activity_pattern_11_weekdaylight_month_1_no_aggregation,activity_pattern_21_weekdaylight_last_12_months_max,location_cluster_02_month_1_no_aggregation,activity_pattern_22_weekend_last_3_months_min,mobility_pattern_05_last_6_months_min,activity_pattern_08_last_9_months_min,activity_pattern_28_last_3_months_mean,device_age_2_month_6_no_aggregation,activity_pattern_11_weekdaynight_month_9_no_aggregation
str,str,f64,f64,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""ab5191f0-5bf7-48d7-b52b-046316…","""2022-04-29""",1.0,1999.0,"""2019-03-01""","""REGION_5""","""TYPE_2""",-184.249,22.0,-224.048,-27.9704,-104.1362,-180.4017,12.7514,234.0074,-414.8524,null,null,-198.5122,-14.5693,254.6177,null,130.2402,null,-172.1113,-67.5607,24.5808,28.0,null,-270.9978,-361.9366,116.7504,-55.6501,154.7281,312.7052,-64.8508,151.8844,…,-134.2671,177.1795,244.5875,3.7521,null,-5.4105,-151.4378,94.8894,-303.3621,-15.1243,-99.0,348.433,-91.2841,null,-252.5821,-35.0847,415.4933,1140.2032,93.1231,null,-299.9051,309.5891,-411.5654,-115.208,89.8558,-382.5563,-281.5171,-115.3785,150.4016,-250.0021,-727.2061,-410.8231,-107.3805,85.0,-19.0,-

## <a id='toc1_3_'></a>[Preprocessing](#toc0_)

### <a id='toc1_3_1_'></a>[Split dataset](#toc0_)

In [171]:
# Select all rows from the set containing training samples. We'll train and validate our model with this data
target = "default"
known_set = df.filter(pl.col(target).is_not_null())

# As the dataset will be splitted in function of time i'm forced to change the schema
known_set = known_set.with_columns(pl.col("credit_application_date").cast(pl.Date()))

# As there are records with almost every value being null i will handle separatedly those clients.
# Null set must be revisited. Separate if more than 80% of values per client are null
alpha = 0.8
null_set = known_set.filter(pl.sum_horizontal(pl.all().is_null()) >= alpha* len(df.columns))
known_set = known_set.filter(pl.sum_horizontal(pl.all().is_null()) < alpha* len(df.columns))

# # Save both sets
# null_set.write_csv("../data/raw/null_set.csv")
# known_set.write_csv("../data/raw/not_null.csv")

In [172]:
# 10 months for train and 4 months for validation
train = known_set.filter(pl.col("credit_application_date")<dt.datetime(2022,7,1))
test = known_set.filter(pl.col("credit_application_date")>=dt.datetime(2022,7,1))

In [173]:
known_set.shape[0], train.shape[0], test.shape[0]

(18348, 11923, 6425)

In [174]:
# Verify that default rates is similar between datasets
train["default"].value_counts(normalize=True), test["default"].value_counts(normalize=True)

(shape: (2, 2)
 ┌─────────┬────────────┐
 │ default ┆ proportion │
 │ ---     ┆ ---        │
 │ f64     ┆ f64        │
 ╞═════════╪════════════╡
 │ 1.0     ┆ 0.160278   │
 │ 0.0     ┆ 0.839722   │
 └─────────┴────────────┘,
 shape: (2, 2)
 ┌─────────┬────────────┐
 │ default ┆ proportion │
 │ ---     ┆ ---        │
 │ f64     ┆ f64        │
 ╞═════════╪════════════╡
 │ 0.0     ┆ 0.80965    │
 │ 1.0     ┆ 0.19035    │
 └─────────┴────────────┘)

In [175]:
# Create training and test sets separating target feature. Transform to pandas for sklearn compatibility
X_train = train.drop(pl.col("default")).to_pandas()
y_train = train.select(target).to_pandas()
X_test = test.drop(pl.col("default")).to_pandas()
y_test = test.select(target).to_pandas()

### <a id='toc1_3_2_'></a>[Feature selection](#toc0_)

#### <a id='toc1_3_2_1_'></a>[Unwanted features](#toc0_)

In [176]:
# Drop uuid cause is unique and credit_applicatio_date to remove time dependency to the predictions
DF_unwanted = DropFeatures(features_to_drop=["uuid", "credit_application_date"])
# fit the transformer
DF_unwanted.fit(X_train)
X_train = DF_unwanted.transform(X_train)


#### <a id='toc1_3_2_2_'></a>[Null rates](#toc0_)

In [177]:
# Retain just features with less than "x (threshold)" rate of null values.
null_threshold = 0.20
null_cols = [col.name for col in (train.drop(pl.col("default")).null_count()/train.shape[0]).select(pl.all().sum() >= null_threshold) if col.all()]
print("Total features filtered with high null values rate: ", len(null_cols))

Total features filtered with high null values rate:  114


In [178]:
DF_nulls = DropFeatures(features_to_drop=null_cols)
# fit the transformer
DF_nulls.fit(X_train)
X_train = DF_nulls.transform(X_train)

#### <a id='toc1_3_2_3_'></a>[Duplicates features](#toc0_)

In [179]:
DDF = DropDuplicateFeatures()
DDF.fit(X_train)
X_train =  DDF.transform(X_train)

#### <a id='toc1_3_2_4_'></a>[Constant and Quasi-constant features](#toc0_)

In [181]:
DCF = DropConstantFeatures(tol = 0.75, missing_values= "ignore")
DCF.fit(X_train)
X_train = DCF.transform(X_train)

#### <a id='toc1_3_2_5_'></a>[Correlated features](#toc0_)

In [184]:
DCORR= DropCorrelatedFeatures(variables=None, method='pearson', threshold=0.8)
DCORR.fit(X_train)
X_train = DCORR.transform(X_train)

In [185]:
X_train

,activity_date_36_month_1_no_aggregation,location_cluster_03_month_1_most_frequent,activity_pattern_35_month_1_no_aggregation,activity_pattern_02_month_12_no_aggregation,device_age_2_month_1_social_mean,activity_pattern_07_last_12_months_max,activity_pattern_25_last_12_months_min,activity_pattern_21_weekdaynight_month_12_no_aggregation,activity_pattern_03_weekend_month_6_no_aggregation,activity_pattern_02_month_1_no_aggregation,...,activity_pattern_27_last_9_months_mean,data_usage_pattern_time_8_month_1_social_min,data_usage_pattern_time_5_month_1_social_mean,activity_pattern_10_month_6_no_aggregation,activity_pattern_10_month_9_no_aggregation,activity_pattern_10_weekend_last_3_months_mean,activity_pattern_11_weekdaylight_month_1_no_aggregation,activity_pattern_22_weekend_last_3_months_min,activity_pattern_08_last_9_months_min,activity_pattern_11_weekdaynight_month_9_no_aggregation
0,2019-03-01,REGION_5,-184.2490,-27.9704,-104.1362,-414.8524,-67.5607,-270.9978,-361.9366,312.7052,...,-99.0000,-91.2841,-35.0847,-411.5654,-115.2080,-281.5171,150.4016,-410.8231,85.0000,207.0453
1,2019-07-01,REGION_6,-171.5127,-49.0834,-103.7082,-414.8524,-59.8400,-7.0000,-245.4993,316.0488,...,194.7235,-90.4938,-34.7007,-364.9055,-112.9766,-281.5171,190.9506,-383.9082,-60.4174,183.0413
2,2002-02-01,REGION_7,-2634.7235,-43.3992,-104.5642,-418.9679,-81.4581,-262.8330,-353.3116,312.7052,...,257.9298,-98.3534,-34.6097,-415.1546,-117.4395,-314.5017,170.6761,-453.1178,85.0000,168.0388
3,2001-01-01,REGION_7,-2634.7235,24.0000,-104.9922,-324.3125,-72.1932,-222.0085,-292.9367,82.0000,...,198.4415,-91.6135,-35.6990,-357.7271,-114.0923,-266.8572,158.0046,-395.4432,-56.0108,204.0448
4,2000-01-01,REGION_7,-2634.7235,-53.9557,-102.4241,-340.7743,-64.4724,-257.3897,-327.4367,402.9812,...,194.7235,-98.6608,-34.8259,-372.0839,-106.8400,-292.5119,201.0878,-445.4279,-58.9486,78.0239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11918,2020-11-01,REGION_7,-135.8509,-43.3992,-99.8560,-172.0408,22.0000,-238.3383,-301.5617,82.0000,...,-99.0000,-96.3556,-35.6933,-318.2456,-65.0000,-156.9082,92.1125,-84.0000,85.0000,-66.0000
11919,2009-02-01,REGION_9,-2634.7235,24.0000,-106.2763,NaN,-75.2815,-140.3597,-172.1870,82.0000,...,-99.0000,-65.0927,-35.5653,-257.2288,-94.5670,-138.5834,84.5096,-84.0000,85.0000,-66.0000
11920,2021-05-01,REGION_4,-120.5672,-27.9704,NaN,-324.3125,22.0000,-7.0000,-38.4998,372.8892,...,-99.0000,NaN,NaN,-153.1413,-103.4928,-131.2534,46.4950,-84.0000,-0.1940,-66.0000
11921,2002-01-01,REGION_7,-2634.7235,24.0000,-101.1401,-357.2361,-90.7230,-175.7409,-146.3120,292.6439,...,224.4677,-56.1573,-36.1227,-235.6935,-100.7035,-189.8929,221.3623,-84.0000,-47.1977,171.0393


### <a id='toc1_3_3_'></a>[Transform](#toc0_)

#### Imputer

### Scaler